<a href="https://colab.research.google.com/github/shivaansh81001/btc-prediction/blob/main/cryptopanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import requests
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

from transformers import pipeline
from textblob import TextBlob

In [45]:
sample=Table().read_table('sample_data/cryptopanic_news.csv')


In [46]:
sample.sort("newsDatetime",descending=False)
sample.show(10)

id,title,sourceId,newsDatetime,url,negative,positive,important,liked,disliked,lol,toxic,saved,comments
873,ApeCoin Community Votes to Stay on Ethereum,2,6/9/2022 6:57,https://cryptopanic.com/news/15444990/ApeCoin-Community- ...,0,0,0,0,0,0,0,0,0
875,Terraform Labs Co-Founder Denies Accusations of Fraud,2,6/9/2022 6:03,https://cryptopanic.com/news/15443520/Terraform-Labs-Co- ...,0,0,0,1,0,0,0,0,0
876,Anchor dev claims he warned Do Kwon over unsustainable 2 ...,7,6/9/2022 5:39,https://cryptopanic.com/news/15442676/Anchor-dev-claims- ...,1,0,1,0,0,0,0,0,0
877,"With 12 Key Measures of Prices, Truflation’s Revamped Da ...",11,6/9/2022 5:32,https://cryptopanic.com/news/15442724/With-12-Key-Measur ...,0,0,0,0,0,0,0,0,0
878,'Father' of the Metaverse Neal Stephenson launching Meta ...,7,6/9/2022 5:28,https://cryptopanic.com/news/15442350/Father-of-the-Meta ...,0,1,0,2,0,0,0,0,0
879,Solana launches $100M fund to attract crypto projects fr ...,14,6/9/2022 5:20,https://cryptopanic.com/news/15442294/Solana-launches-10 ...,0,1,0,1,1,0,0,0,0
880,Alibaba Cloud launches service to build NFT marketplaces,13,6/9/2022 5:01,https://cryptopanic.com/news/15442278/Alibaba-Cloud-laun ...,0,2,0,1,0,1,0,0,0
881,"Balderton, Animoca Brands back Request Finance in $5.5 m ...",3,6/9/2022 4:45,https://cryptopanic.com/news/15442253/Balderton-Animoca- ...,0,0,0,0,0,0,0,0,0
882,Optimism Foundation sends $20M to the wrong wallet – OP ...,14,6/9/2022 4:00,https://cryptopanic.com/news/15442200/Optimism-Foundatio ...,1,0,1,3,0,3,0,0,3
883,Creating Utility for Tokenized Derivatives of Proof-of-S ...,10,6/9/2022 3:40,https://cryptopanic.com/news/15442162/Creating-Utility-f ...,0,0,1,0,0,0,0,0,0


In [47]:
def format_date_time(dt):
  temp = dt.split()
  return '-'.join(temp[0].split('/')[::-1])+' '+temp[1]+':00'

In [48]:
#test

format_date_time('6/9/2022 5:20')

'2022-9-6 5:20:00'

In [49]:
new_date= sample.apply(format_date_time,"newsDatetime")
sample= sample.with_column('newsDatetime',new_date)


In [50]:
sample= sample.sort("newsDatetime",descending=True)
sample.show(10)

id,title,sourceId,newsDatetime,url,negative,positive,important,liked,disliked,lol,toxic,saved,comments
169651,Quant price is poised for a rally as on-chain data and p ...,157,2024-9-9 9:55:00,https://cryptopanic.com/news/19882813/Quant-price-is-poi ...,0,0,0,0,0,0,0,0,0
173505,Top AI Cryptos To Buy This Week As Apple Pushes iPhone 1 ...,144,2024-9-9 9:38:00,https://cryptopanic.com/news/19882483/Top-AI-Cryptos-To- ...,0,0,0,0,0,1,0,0,0
169708,Bitcoin Layer 2 Protocol Stacks Achieves New High in Sma ...,24,2024-9-9 9:30:00,https://cryptopanic.com/news/19882448/Bitcoin-Layer-2-Pr ...,0,0,0,0,0,0,0,0,0
170772,Starknet minting vote will be gasless with new governanc ...,7,2024-9-9 9:30:00,https://cryptopanic.com/news/19882450/Starknet-minting-v ...,0,0,0,0,0,0,0,0,0
170773,"Snapshot, Popular DAO Voting Platform, Finally Moves On- ...",30,2024-9-9 9:30:00,https://cryptopanic.com/news/19882480/Snapshot-Popular-D ...,0,0,0,0,0,0,0,0,0
172324,Snapshot X launches to enable gas-free onchain voting vi ...,131,2024-9-9 9:30:00,https://cryptopanic.com/news/19882434/Snapshot-X-launche ...,0,0,0,0,0,0,0,0,0
171134,Terra Luna Classic To Burn 1 Billion USTC & 275 Billion ...,139,2024-9-9 9:25:00,https://cryptopanic.com/news/19882427/Terra-Luna-Classic ...,0,0,0,0,0,0,0,0,0
172072,WazirX Hacker Moves $11M Stolen Ether to Tornado Cash,30,2024-9-9 9:25:00,https://cryptopanic.com/news/19882479/WazirX-Hacker-Move ...,0,0,0,0,0,0,0,0,0
172945,Brett Price Prediction September: Brett & Pepe Plummets ...,5,2024-9-9 9:24:00,https://cryptopanic.com/news/19882426/Brett-Price-Predic ...,0,0,0,0,0,0,0,0,0
168883,"HNT Rallies 16.63% Defying All Odds, Will it Catch Up Mp ...",5,2024-9-9 9:18:00,https://cryptopanic.com/news/19882408/HNT-Rallies-1663-D ...,0,0,0,0,0,0,0,0,0


In [51]:
sample =sample.where("title",are.containing("Bitcoin")).select('title','newsDatetime')
sample.show(10)

title,newsDatetime
Bitcoin Layer 2 Protocol Stacks Achieves New High in Sma ...,2024-9-9 9:30:00
Bitcoin Investors Withdraw $728M,2024-9-9 6:46:00
"Unisat-backed Fractal Bitcoin goes live on mainnet, taps ...",2024-9-9 6:27:00
"Bitcoin Price Rebounds, But Could the Downtrend Return?",2024-9-9 2:44:00
Trump Victory Could Boost Bitcoin to $90K — Harris Win M ...,2024-9-9 23:30:00
"Bitcoin Leveraged Positions Building Up: BTC To $50,000 ...",2024-9-9 22:00:00
"FTM, AAVE, TAO see double-digit gains as Bitcoin retests ...",2024-9-9 20:35:00
Bitcoin To Break $100K By 2025? Experts Weigh In On Pote ...,2024-9-9 20:30:00
"Bitcoin, Ethereum, Dogecoin Resume Uptrend: 'BTC Needs T ...",2024-9-9 20:00:00
Bitcoin Cash Up More Than 6% In 24 hours,2024-9-9 19:00:00


In [52]:
sentiment_analyzer = pipeline("sentiment-analysis", model="ProsusAI/finbert")

Device set to use cuda:0


In [53]:
sentiment_=[]
polarity_=[]
subjectivity_=[]

In [54]:
for news in sample.column("title"):
  result=sentiment_analyzer(news)[0]
  ans =result['label']

  if ans =="positive":
    sentiment_.append(1)
  elif ans =="negative":
    sentiment_.append(-1)
  elif ans =="neutral":
    sentiment_.append(0)
  else:
    print(ans)

  blob=TextBlob(news)

  polarity_.append(round(blob.sentiment.polarity,2))
  subjectivity_.append(round(blob.sentiment.subjectivity,2))

In [55]:
print(len(sentiment_),len(polarity_),len(subjectivity_))

11025 11025 11025


In [56]:
sample= sample.with_columns("sentiment",np.array(sentiment_),"polarity",np.array(polarity_),"subjectivity",np.array(subjectivity_))
sample.show(3)

title,newsDatetime,sentiment,polarity,subjectivity
Bitcoin Layer 2 Protocol Stacks Achieves New High in Sma ...,2024-9-9 9:30:00,1,0.17,0.55
Bitcoin Investors Withdraw $728M,2024-9-9 6:46:00,-1,0,0
"Unisat-backed Fractal Bitcoin goes live on mainnet, taps ...",2024-9-9 6:27:00,0,0.14,0.5
